# <img src="https://stage.sunsolve.com/_astro/SunSolveLogo.BnIkxJwG_ZSGGx1.svg">
# P90 Analysis Tool
This notebook provides a tool for simulating uncertainty in PV systems using the PV Lighthouse 'P90' client.

In [16]:
import os, sys
USING_COLAB = False  # Set to True if using Google Colab and Drive (Colab doesn't save files unless using Drive)
REQUIRED_FILES = ["Data", "SunSolveLogo.svg", "README.md",
    "pvl_p90_client-0.1.0.156+dev-py3-none-any.whl", "UncertaintyFunctions.ipynb"]

# Set up Drive folder if using Colab + Drive (must enter folder path below)
if USING_COLAB and "google.colab" in sys.modules:
    # Change this to wherever you have saved this file on your Google Drive
    folder = "/content/G/MyDrive/Colab Notebooks/Uncertainty Colabs"    # G:\\ will be handled below
    folder = folder.replace("G://", "/content/G/").replace("G:\\\\", "\\content\\G\\").replace("\\","/")
    from google.colab import drive
    drive.mount("G")   # Mount Google Drive to "content/G" (Will ask for authorization)
    # Change the working directory to wherever you have saved this file on your drive
    os.chdir(folder)
# Otherwise, we keep the current directory.

# Check if the required files/folders are available, or clone them into a "SunSolve P90" folder
if not all(i in os.listdir() for i in REQUIRED_FILES):
    if not "SunSolve P90" in os.listdir() or not all(i in os.listdir("SunSolve P90") for i in REQUIRED_FILES):
        os.makedirs("SunSolve P90", exist_ok=True)
        os.system("git clone --depth 1 https://github.com/SF-PVL/P90-Notebook.git 'SunSolve P90'")
    os.chdir("SunSolve P90")

# Install the P90 client (Colab: the pip dependency error can be ignored)
%pip install pvl_p90_client-0.1.0.156+dev-py3-none-any.whl --quiet

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Load helper file functions, log in to your PV Lighthouse account and start running a test simulation
%run "UncertaintyFunctions.ipynb"   # If this fails, try restarting the kernel and re-running this cell

# Set uncertainty simulation constants (time scales with N_SIMS*N_YEARS, 5000*5 ≈ 120 seconds)
N_SIMS = 1000
N_YEARS = 12
simulation_options = build_simulation_options(number_of_years=N_YEARS, number_of_simulations=N_SIMS)

# Set P-value preferences
p_values = [50, 90, 95]     # e.g. [50, 90, 95] -> P50, P90, P95
p_min = 0.75                # The minimum fraction of P50 to be binned on histograms
p_delta = 0.01              # The bin size for histograms
result_options = build_result_options(p_min=p_min, p_delta=p_delta, p_values=p_values)

# Collect weather data: sydney.pvw file is provided as part of the source
weather_file_path = "Data/sydney.pvw"
weather_data = load_weather_data_from_pvw_file(weather_file_path)

# Set up system
module_info = build_module_info(length=2.0, width=1.0, bifaciality=0.8)

system_info = build_system_info(
    modules_per_string=1,
    fallback_module_tilt_in_degrees=30,
    number_of_inverters=1,
    num_strings_per_inverter=1,
    row_pitch_in_m=5.6,
    azimuth_in_degrees=90            # Direction modules face for positive tilt. 0=North, 90=East, 180=South
)

electrical_settings = build_electrical_settings(
    inverter_efficiency=0.98,
    module_to_module_mismatch=0.01,
    string_wiring_loss=0.01,
    inverter_wiring_loss=0.01,
    max_power_tracking_loss=0.0
)

thermal_settings = build_thermal_settings(uc=25, uv=1.2, alpha=0.9)

optical_settings = build_optical_settings(fallback_albedo=0.2,
    fallback_soiling_front=0.02, fallback_soiling_rear=0.002)

operational_settings = build_operational_settings(annual_degradation_rate=0.005,
    curtailment=0.02, availability=0.98)

2025-10-09 16:24:34,292 - INFO - Connected to P90 service at middleware.pvlighthouse.com.au


Connecting to PV Lighthouse...
Authenticating...
Authentication successful!
Loading time step data...


2025-10-09 16:24:34,489 - INFO - P90 client connection closed


Loaded 8760 weather data points


In [18]:
# View irradiance data for specific day
interactive_weather_plot(weather_file_path, weather_data)

interactive(children=(IntSlider(value=1, description='Day of year:', max=365, min=1), Output()), _dom_classes=…

## Set Up Modifiers to Introduce Uncertainty

Set up input distributions (probability density functions applied to inputs)

**Target Inputs**: ('DistributionInput.____')

GHI, DiffuseFraction, WindSpeed, Temperature, ModulePower, SpectralCorrection, SoilingFront, SoilingRear, Uc, Uv, Alpha, AnnualDegradationRate, Availability, YieldModifier, CircumsolarFraction, UndulatingGround, ExtraIrradiance, Curtailment, DCHealth, InverterToInverterMismatch, StringToStringMismatch, ModuleToModuleMismatch, CellToCellMismatch, InverterEfficiency, ModuleEfficiencyTemperatureCoefficient, Albedo

**simToSim**: sim-to-sim variability, affecting all years for that simulation; **yearToYear**: year-to-year variability; **stepToStep**: hour-to-hour variability

Note: AnnualDegradationRate is a rate/year and should be positive, soiling should be positive.

In [19]:
# Interactive probability density function (pdf) viewer, to help define your distributions
interactive_distribution_plot()

In [20]:
distribution_list = [
    create_distribution(DistributionInput.AnnualDegradationRate, simToSim=["SkewedGaussian", 6, 1, 0.002], yearToYear=["Gaussian", 1, 0.02]),
    create_distribution(DistributionInput.GHI, simToSim=["Gaussian", 1, 0.06], yearToYear=["Gaussian", 1, 0.04], stepToStep=["SkewedGaussian", -2, 0.95, 0.05]),
    create_distribution(DistributionInput.Temperature, simToSim=["Gaussian", 1, 0.06], yearToYear=["Gaussian", 1, 0.04], stepToStep=["SkewedGaussian", -2, 0.95, 0.05]),
    create_distribution(DistributionInput.SoilingFront, simToSim=["Weibull", 0, 0.04, 2], yearToYear=["Gaussian", 0.08, 0.04]),
]

## Send Request and Start Analysis

In [21]:
# Build and send request
p90_request = build_request(
    time_step_data=weather_data,
    module=module_info,
    system=system_info,
    electrical=electrical_settings,
    optical=optical_settings,
    thermal=thermal_settings,
    operational=operational_settings,
    distributions=distribution_list,
    simulation_options=simulation_options,
    result_options=result_options
)

summary, used_inputs = request_analysis(p90_request)

2025-10-09 16:24:35,958 - INFO - Connected to P90 service at middleware.pvlighthouse.com.au
2025-10-09 16:24:35,975 - INFO - Starting P90 analysis request


Starting uncertainty analysis...


2025-10-09 16:24:44,923 - INFO - === P90 Analysis Inputs Used ===
2025-10-09 16:24:44,924 - INFO - Module: Length=2.000m, Width=1.000m, HeightAboveGround=1.500m, PowerRatingAtSTC=460.0W, CellToCellMismatch=0.0040, EfficiencyTempCoeff=0.002950, Bifaciality=0.800
2025-10-09 16:24:44,925 - INFO - System: ModulesPerString=1, StringsPerInverter=1, NumberOfInverters=1
2025-10-09 16:24:44,927 - INFO - System Geometry: RowPitch=5.60m, ModuleAzimuth=90.0°, FallbackModuleTilt=30.0°
2025-10-09 16:24:44,930 - INFO - System Tracking: TrackingCalculation=0, TiltLimit=0.0°
2025-10-09 16:24:44,932 - INFO - Electrical Efficiencies: InverterEff=0.9800, ModuleToModuleMismatch=0.0100, StringWiringLoss=0.0100
2025-10-09 16:24:44,937 - INFO - Electrical Losses: MaxPowerTracking=0.0000, InverterWiring=0.0100, StringToStringMismatch=0.0000, InverterToInverterMismatch=0.0000
2025-10-09 16:24:44,939 - INFO - Optical Multipliers: BeamFront=1.000, BeamRear=1.000, IsotropicFront=1.010, IsotropicRear=1.000
2025-10-

Progress: 100.00%



2025-10-09 16:25:10,696 - INFO - Received P90 analysis summary


Analysis complete! Summary contains 36 yearly P-values


2025-10-09 16:25:10,770 - INFO - P90 client connection closed


Request took 35 s, running 12000 simulations in total (345 sims/s)


## Results
Results are expressed in terms of 'P50 deviation' (normalised yield), where a value of 1 is the P50 yield for that year.

A P90 might have a P50 deviation of '0.88' - this means that 90% of simulated yields were at least 88% of the P50 yield.

The P50 yields of each year are also compared, where the P50 deviation is displayed relative to the year 1 P50 yield.

[Useful reading](https://apvi.org.au/solar-research-conference/wp-content/uploads/2024/12/McIntosh_K_Asymmetry_in_the_energy_yield_forecasts_for_PV_power_plants.pdf)

Absolute yield estimates are not provided, but [SunSolve Yield](https://sunsolve.info/yield/) can be used to estimate this accurately. You can also supply an absolute yield estimate for year one below, which will then scale all of the relative yields to absolute yields with this year 1 P50 value as the basis. It is assumed to be in 'MWh' if so, but can be changed by supplying a 'yield_units' string to the plotting functions.

Note: The first and last bins of the histograms are larger than the others, including all values to their left and right, respectively.

In [22]:
year_one_yield = 1              # Change this to plot absolute yields, if you know what the first year yield should be in MWh
plot_yearly_P50_Values(summary, year_one_yield=year_one_yield)
plot_interactive_histogram(summary, p_min=p_min, p_delta=p_delta, year_one_yield=year_one_yield)

interactive(children=(IntSlider(value=1, description='Year:', max=12, min=1), Output()), _dom_classes=('widget…

In [29]:
plot_pvalues(summary, year_one_yield=year_one_yield)

## Export your results

In [30]:
date_prefix = datetime.date.today().strftime("%y%m%d_")
filename = date_prefix + f"P90 summary_{N_YEARS}x{N_SIMS}"
export_to_excel(summary, filename, p_min=p_min, p_delta=p_delta, used_inputs=used_inputs, dist_list=distribution_list)

Summary data saved to '251009_P90 summary_12x1000.xlsx'


## Further analysis

Continue your analysis below